In [62]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import constants
from scipy import stats
from scipy.optimize import leastsq

In [63]:
k = constants.value('Boltzmann constant in eV/K')
e = np.exp(1)

In [64]:
GRDF = pd.read_csv("../Results/sorted_data.csv")
GRDF = GRDF.drop(['Unnamed: 1'], axis = 1)

In [100]:
def f(group):

    split = np.argmax(group.reset_index().STVlogged)  # split
    x     = group.reset_index().adjTemp
    y     = group.reset_index().STVlogged
    xVals = group.reset_index().UsedTemp

    if split + 1 == len(y) or split == 0\
                           or split == 1\
                           or x[:split].nunique() == 1\
                           or x[split:].nunique() == 1:
        
        lm1 = stats.linregress(x, y)
        lm2 = stats.linregress(x, y)
    else:
        try:
            lm1 = stats.linregress(x[:split], y[:split])
        except ValueError:
            lm1 = stats.linregress(x, y)
        try:
            lm2 = stats.linregress(x[split:], y[split:])
        except ValueError:
            lm2 = stats.linregress(x, y)
        
    vals = {"E"     : lm1[0],
            "El"    : lm1[0]/2,
            "Eh"    : lm2[0],
            "Eint"  : lm1[1],
            "Ehint" : lm2[1],
            "B0"    : lm1[0]*(1/(k*283.15)) + lm1[1],
            "Th"    : xVals[np.argmax(y)],
            "Tl"    : min(xVals)}
    
    return pd.DataFrame(vals, index = [0])

strt_vals = GRDF.groupby("NewID").apply(f)

In [104]:
strt_vals.to_csv("../Results/starting_vals.csv")